## Input data and placeholders

The code below sets up the input data and placeholders for the classifier.

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Python optimisation variables
learning_rate = 0.0001
epochs = 10
batch_size = 50

# declare the training data placeholders
# input x - for 28 x 28 pixels = 784 - this is the flattened image data that is drawn from 
# mnist.train.nextbatch()
x = tf.placeholder(tf.float32, [None, 784])
# dynamically reshape the input
x_shaped = tf.reshape(x, [-1, 28, 28, 1])
# now declare the output data placeholder - 10 digits
y = tf.placeholder(tf.float32, [None, 10])

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


TensorFlow has a handy loader for the MNIST data which is sorted out in the first couple of lines.  After that we have some variable declarations which determine the optimisation behaviour (learning rate, batch size etc.).  Next, we declare a placeholder (see this tutorial for explanations of placeholders) for the image input data, x.  The image input data will be extracted using the mnist.train.nextbatch() function, which supplies a flattened 28×28=784 node, single channel greyscale representation of the image. However, before we can use this data in the TensorFlow convolution and pooling functions, such as conv2d() and max_pool() we need to reshape the data as these functions take 4D data only.

The format of the data to be supplied is [i, j, k, l] where i is the number of training samples, j is the height of the image, k is the weight and l is the channel number.  Because we have a greyscale image, l will always be equal to 1 (if we had an RGB image, it would be equal to 3).  The MNIST images are 28 x 28, so both j and k are equal to 28.  When we reshape the input data x into x_shaped, theoretically we don’t know the size of the first dimension of x, so we don’t know what i is.  However, tf.reshape() allows us to put -1 in place of i and it will dynamically reshape based on the number of training samples as the training is performed.  So we use [-1, 28, 28, 1] for the second argument in tf.reshape().

Finally, we need a placeholder for our output training data, which is a [?, 10] sized tensor – where the 10 stands for the 10 possible digits to be classified.  We will use the mnist.train.next_batch() to extract the digits labels as a one-hot vector – in other words, a digit of “3” will be represented as [0, 0, 0, 1, 0, 0, 0, 0, 0, 0].

## Defining the convolution layers

Because we have to create a couple of convolutional layers, it’s best to create a function to reduce repetition:

In [3]:
def create_new_conv_layer(input_data, num_input_channels, num_filters, filter_shape, pool_shape, name):
    # setup the filter input shape for tf.nn.conv_2d
    conv_filt_shape = [filter_shape[0], filter_shape[1], num_input_channels,
                      num_filters]

    # initialise weights and bias for the filter
    weights = tf.Variable(tf.truncated_normal(conv_filt_shape, stddev=0.03),
                                      name=name+'_W')
    bias = tf.Variable(tf.truncated_normal([num_filters]), name=name+'_b')

    # setup the convolutional layer operation
    out_layer = tf.nn.conv2d(input_data, weights, [1, 1, 1, 1], padding='SAME')

    # add the bias
    out_layer += bias

    # apply a ReLU non-linear activation
    out_layer = tf.nn.relu(out_layer)

    # now perform max pooling
    ksize = [1, pool_shape[0], pool_shape[1], 1]
    strides = [1, 2, 2, 1]
    out_layer = tf.nn.max_pool(out_layer, ksize=ksize, strides=strides, 
                               padding='SAME')

    return out_layer

I’ll step through each line/block of this function below:

In [4]:
'conv_filt_shape = [filter_shape[0], filter_shape[1], num_input_channels, num_filters]'

'conv_filt_shape = [filter_shape[0], filter_shape[1], num_input_channels, num_filters]'

This line sets up a variable to hold the shape of the weights that determine the behaviour of the 5×5 convolutional filter.  The format that the conv2d() function receives for the filter is: [filter_height, filter_width, in_channels, out_channels].  The height and width of the filter are provided in the filter_shape variables (in this case [5, 5]).  The number of input channels, for the first convolutional layer is simply 1, which corresponds to the single channel greyscale MNIST image.  However, for the second convolutional layer it takes the output of the first convolutional layer, which has a 32 channel output.  Therefore, for the second convolutional layer, the input channels is 32.  As defined in the block diagram above, the number of output channels of the first layer is 32, and for the second layer it is 64.

In [5]:
'''# initialise weights and bias for the filter
weights = tf.Variable(tf.truncated_normal(conv_filt_shape, stddev=0.03),
                                  name=name+'_W')
bias = tf.Variable(tf.truncated_normal([num_filters]), name=name+'_b')'''

"# initialise weights and bias for the filter\nweights = tf.Variable(tf.truncated_normal(conv_filt_shape, stddev=0.03),\n                                  name=name+'_W')\nbias = tf.Variable(tf.truncated_normal([num_filters]), name=name+'_b')"

In these lines we create the weights and bias for the convolutional filter and randomly initialise the tensors.  If you need to brush up on these concepts, check out this tutorial.

In [6]:
'''# setup the convolutional layer operation
out_layer = tf.nn.conv2d(input_data, weights, [1, 1, 1, 1], padding='SAME')'''

"# setup the convolutional layer operation\nout_layer = tf.nn.conv2d(input_data, weights, [1, 1, 1, 1], padding='SAME')"

This line is where we setup the convolutional filter operation.  The variable input_data is self-explanatory, as are the weights.  The size of the weights tensor show TensorFlow what size the convolutional filter should be.  The next argument [1, 1, 1, 1] is the strides parameter that is required in conv2d().  In this case, we want the filter to move in steps of 1 in both the x and y directions (or height and width directions).  This information is conveyed in the strides[1] and strides[2] values – both equal to 1 in this case.  The first and last values of strides are always equal to 1, if they were not, we would be moving the filter between training samples or between channels, which we don’t want to do.  The final parameter is the padding.  Padding determines the output size of each channel and when it is set to “SAME” it produces dimensions of:

out_height = ceil(float(in_height) / float(strides[1]))
out_width  = ceil(float(in_width) / float(strides[2]))

For the first convolutional layer, in_height = in_width = 28, and strides[1] = strides[2] = 1.  Therefore the padding of the input with 0.0 nodes will be arranged so that the out_height = out_width = 28 – there will be no change in size of the output.  This padding is to avoid the fact that, when traversing a (x,y) sized image or input with a convolutional filter of size (n,m), with a stride of 1 the output would be (x-n+1,y-m+1).  So in this case, without padding, the output size would be (24,24).  We want to keep the sizes of the outputs easy to track, so we chose the “SAME” option as the padding so we keep the same size.

In [7]:
'''# add the bias
out_layer += bias
# apply a ReLU non-linear activation
out_layer = tf.nn.relu(out_layer)'''

'# add the bias\nout_layer += bias\n# apply a ReLU non-linear activation\nout_layer = tf.nn.relu(out_layer)'

In the two lines above, we simply add a bias to the output of the convolutional filter, then apply a ReLU non-linear activation function.

In [8]:
'''# now perform max pooling
ksize = [1, pool_shape[0], pool_shape[1], 1]
strides = [1, 2, 2, 1]
out_layer = tf.nn.max_pool(out_layer, ksize=ksize, strides=strides, 
                               padding='SAME')

return out_layer'''

"# now perform max pooling\nksize = [1, pool_shape[0], pool_shape[1], 1]\nstrides = [1, 2, 2, 1]\nout_layer = tf.nn.max_pool(out_layer, ksize=ksize, strides=strides, \n                               padding='SAME')\n\nreturn out_layer"

The max_pool() function takes a tensor as its first input over which to perform the pooling.  The next two arguments ksize and strides define the operation of the pooling.  Ignoring the first and last values of these vectors (which will always be set to 1), the middle values of ksize (pool_shape[0] and pool_shape[1]) define the shape of the max pooling window in the x and y directions.  In this convolutional neural networks example, we are using a 2×2 max pooling window size.  The same applies with the strides vector – because we want to down-sample, in this example we are choosing strides of size 2 in both the x and y directions (strides[1] and strides[2]). This will halve the input size of the (x,y) dimensions.

Finally, we have another example of a padding argument.  The same rules apply for the ‘SAME’ option as for the convolutional function conv2d().  Namely:

out_height = ceil(float(in_height) / float(strides[1]))
out_width  = ceil(float(in_width) / float(strides[2]))

Punching in values of 2 for strides[1] and strides[2] for the first convolutional layer we get an output size of (14, 14).  This is a halving of the input size (28, 28), which is what we are looking for.  Again, TensorFlow will organise the padding so that this output shape is what is achieved, which makes things nice and clean for us.

Finally we return the out_layer object, which is actually a sub-graph of its own, containing all the operations and weight variables within it.  We create the two convolutional layers in the main program by calling the following commands:

In [9]:
# create some convolutional layers
layer1 = create_new_conv_layer(x_shaped, 1, 32, [5, 5], [2, 2], name='layer1')
layer2 = create_new_conv_layer(layer1, 32, 64, [5, 5], [2, 2], name='layer2')

As you can see, the input to layer1 is the shaped input x_shaped and the input to layer2 is the output of the first layer.  Now we can move on to creating the fully connected layers.

## The fully connected layers

As previously discussed, first we have to flatten out the output from the final convolutional layer.  It is now a 7×7 grid of nodes with 64 channels, which equates to 3136 nodes per training sample.  We can use tf.reshape() to do what we need:

In [12]:
flattened = tf.reshape(layer2, [-1, 7 * 7 * 64])

Again, we have a dynamically calculated first dimension (the -1 above), corresponding to the number of input samples in the training batch. Next we setup the first fully connected layer:

In [13]:
# setup some weights and bias values for this layer, then activate with ReLU
wd1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1000], stddev=0.03), name='wd1')
bd1 = tf.Variable(tf.truncated_normal([1000], stddev=0.01), name='bd1')
dense_layer1 = tf.matmul(flattened, wd1) + bd1
dense_layer1 = tf.nn.relu(dense_layer1)

If the above operations are unfamiliar to you, please check out my previous TensorFlow tutorial.  Basically we are initialising the weights of the fully connected layer, multiplying them with the flattened convolutional output, then adding a bias.  Finally, a ReLU activation is applied.  The next layer is defined by:

In [14]:
# another layer with softmax activations
wd2 = tf.Variable(tf.truncated_normal([1000, 10], stddev=0.03), name='wd2')
bd2 = tf.Variable(tf.truncated_normal([10], stddev=0.01), name='bd2')
dense_layer2 = tf.matmul(dense_layer1, wd2) + bd2
y_ = tf.nn.softmax(dense_layer2)

This layer connects to the output, and therefore we use a soft-max activation to produce the predicted output values y_.  We have now defined the basic structure of our convolutional neural network.  Let’s now define the cost function.

## The cross-entropy cost function

We could develop our own cross-entropy cost expression, as we did in the previous TensorFlow tutorial, based on the value y_.   However, then we have to be careful about handling NaN values.  Thankfully, TensorFlow provides a handy function which applies soft-max followed by cross-entropy loss:

In [15]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=dense_layer2, labels=y))

The function softmax_cross_entropy_with_logits() takes two arguments – the first (logits) is the output of the matrix multiplication of the final layer (plus bias) and the second is the training target vector.  The function first takes the soft-max of the matrix multiplication, then compares it to the training target using cross-entropy.  The result is the cross-entropy calculation per training sample, so we need to reduce this tensor into a scalar (a single value).  To do this we use tf.reduce_mean() which takes a mean of the tensor.

## The training of the convolutional neural network

The following code is the remainder of what is required to train the network.  It is a replication of what is explained in my previous TensorFlow tutorial, so please refer to that tutorial if anything is unclear.  We’ll be using mini-batches to train our network.  The essential structure is:

- Create an optimiser
- Create correct prediction and accuracy evaluation operations
- Initialise the operations
- Determine the number of batch runs within an training epoch
- For each epoch:
     - For each batch:
         - Extract the batch data
         - Run the optimiser and cross-entropy operations
         - Add to the average cost
     - Calculate the current test accuracy
     - Print out some results
- Calculate the final test accuracy and print

The code to execute this is:

In [ ]:
# add an optimiser
optimiser = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# setup the initialisation operator
init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    # initialise the variables
    sess.run(init_op)
    total_batch = int(len(mnist.train.labels) / batch_size)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
            _, c = sess.run([optimiser, cross_entropy], 
                            feed_dict={x: batch_x, y: batch_y})
            avg_cost += c / total_batch
        test_acc = sess.run(accuracy, 
                       feed_dict={x: mnist.test.images, y: mnist.test.labels})
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost), 
                 "test accuracy: {:.3f}".format(test_acc))

    print("\nTraining complete!")
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels}))

The final code can be found on this site’s GitHub repository.  Note the final code on that repository contains some TensorBoard visualisation operations, which have not been covered in this tutorial and will have a dedicated future article to explain.

Caution:  This is a relatively large network and on a standard home computer is likely to take at least 10-20 minutes to run.

## The results

As can be observed, after 10 epochs we have reached an impressive prediction accuracy of 99%.  This result has been achieved without extensive optimisation of the convolutional neural network’s parameters, and also without any form of regularisation.  This is compared to the best accuracy we could achieve in our standard neural network ~98% – as can be observed in the previous tutorial.

The accuracy difference will be even more prominent when comparing standard neural networks with convolutional neural networks on more complicated data-sets, like the CIFAR data.  However, that is a topic for another day.  Have fun using TensorFlow and convolutional neural networks!  By the way, if you want to see how to build a neural network in Keras, a more stream-lined framework, check out my Keras tutorial. Also, if you’d like to explore more deep learning architectures in TensorFlow, check out my recurrent neural networks and LSTM tutorial.